In [25]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
from torch import nn, optim
from torch.autograd import Variable
from gpytorch.kernels import RBFKernel, GridInterpolationKernel
from gpytorch.means import ConstantMean
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.random_variables import GaussianRandomVariable
from torch.utils.data import TensorDataset, DataLoader

%matplotlib inline

In [102]:
import pandas as pd
data = pd.read_csv('CASP.csv')
y_data = Variable(torch.from_numpy(data['RMSD'].as_matrix()).float())
x_data = Variable(torch.from_numpy(data[['F%d' % i for i in range(1, 10)]].as_matrix()).float())
permutation = torch.randperm(len(y_data))
n_train = int(len(permutation) * 0.9)
train_indices = permutation[:n_train]
test_indices = permutation[n_train:]

x_train = x_data[train_indices]
y_train = y_data[train_indices]
x_test = x_data[test_indices]
y_test = y_data[test_indices]

In [47]:
train_dataset = TensorDataset(x_train.data, y_train.data)
train_loader = DataLoader(train_dataset, batch_size=64)

In [48]:
def calc_error(model):
    predictions = model(x_test)
    return (predictions - y_test).abs().mean()

In [98]:
class DeepNet(nn.Sequential):
    def __init__(self):
        super(DeepNet, self).__init__(
            nn.Linear(9, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(True),
            nn.Linear(100, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(True),
            nn.Linear(100, 2),
            nn.BatchNorm1d(2),
            nn.ReLU(True),
            nn.Linear(2, 1)
        )
        
deep_net = DeepNet()
# Optimize the model
deep_net.train()
criterion = nn.MSELoss()
optimizer = optim.Adam(deep_net.parameters(), lr=0.01, weight_decay=1e-3)
optimizer.n_iter = 0
for i in range(5):
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = deep_net(Variable(x_batch))
        loss = criterion(output, Variable(y_batch.float()))
        loss.backward()
        optimizer.n_iter += 1
        print('Epoch %d/3 - Loss: %.3f' % (i + 1, loss.data[0]))
        optimizer.step()
deep_net.eval()

calc_error(deep_net)

Epoch 1/3 - Loss: 86.601
Epoch 1/3 - Loss: 115.958
Epoch 1/3 - Loss: 101.756
Epoch 1/3 - Loss: 97.524
Epoch 1/3 - Loss: 87.088
Epoch 1/3 - Loss: 119.770
Epoch 1/3 - Loss: 77.887
Epoch 1/3 - Loss: 126.890
Epoch 1/3 - Loss: 91.046
Epoch 1/3 - Loss: 92.814
Epoch 1/3 - Loss: 93.203
Epoch 1/3 - Loss: 103.587
Epoch 1/3 - Loss: 92.904
Epoch 1/3 - Loss: 126.100
Epoch 1/3 - Loss: 108.455
Epoch 1/3 - Loss: 86.821
Epoch 1/3 - Loss: 100.499
Epoch 1/3 - Loss: 98.467
Epoch 1/3 - Loss: 98.619
Epoch 1/3 - Loss: 111.510
Epoch 1/3 - Loss: 83.825
Epoch 1/3 - Loss: 112.012
Epoch 1/3 - Loss: 105.253
Epoch 1/3 - Loss: 88.490
Epoch 1/3 - Loss: 92.927
Epoch 1/3 - Loss: 131.465
Epoch 1/3 - Loss: 98.968
Epoch 1/3 - Loss: 114.377
Epoch 1/3 - Loss: 102.736
Epoch 1/3 - Loss: 105.962
Epoch 1/3 - Loss: 86.953
Epoch 1/3 - Loss: 72.104
Epoch 1/3 - Loss: 102.567
Epoch 1/3 - Loss: 85.080
Epoch 1/3 - Loss: 98.200
Epoch 1/3 - Loss: 68.004
Epoch 1/3 - Loss: 76.163
Epoch 1/3 - Loss: 93.726
Epoch 1/3 - Loss: 94.516
Epoch 1/3

Variable containing:
 13.5162
[torch.FloatTensor of size 1]

In [99]:
nn.Sequential(*list(deep_net.modules())[1:9])

Sequential (
  (0): Linear (9 -> 100)
  (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU (inplace)
  (3): Linear (100 -> 100)
  (4): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True)
  (5): ReLU (inplace)
  (6): Linear (100 -> 2)
  (7): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True)
)

In [104]:
class DklGPModel(gpytorch.GPModel):
    def __init__(self, deep_net):
        super(DklGPModel, self).__init__(GaussianLikelihood(log_noise_bounds=(-5, 5)))
        self.mean_module = ConstantMean(constant_bounds=(-1, 1))
        self.deep_net = deep_net

        covar_module = RBFKernel(log_lengthscale_bounds=(-5, 5))
        self.grid_covar_module = GridInterpolationKernel(covar_module)
        self.register_parameter('log_outputscale', nn.Parameter(torch.Tensor([0])), bounds=(-1, 1))
        self.initialize_interpolation_grid(20, grid_bounds=[(-3, 2), (-3, 2)])
        covar_module.initialize(log_lengthscale=5)

    def forward(self, x):
        mean_x = self.mean_module(x)
        features = self.deep_net(x) / 10.0
        covar_x = self.grid_covar_module(features)
        covar_x = covar_x.mul(self.log_outputscale.exp())
        return GaussianRandomVariable(mean_x, covar_x)
    
model = DklGPModel(nn.Sequential(*list(deep_net.modules())[1:9]))
model.condition(x_train, y_train)

DklGPModel (
  (likelihood): GaussianLikelihood (
  )
  (mean_module): ConstantMean (
  )
  (deep_net): Sequential (
    (0): Linear (9 -> 100)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): Linear (100 -> 100)
    (4): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU (inplace)
    (6): Linear (100 -> 2)
    (7): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True)
  )
  (grid_covar_module): GridInterpolationKernel (
    (base_kernel_module): RBFKernel (
    )
  )
)

In [105]:
# Optimize the model
model.train()
optimizer = optim.Adam(model.parameters(), lr=0.1)
optimizer.n_iter = 0
for i in range(30):
    optimizer.zero_grad()
    output = model(x_train)
    loss = -model.marginal_log_likelihood(output, y_train)
    loss.backward()
    optimizer.n_iter += 1
    print('Iter %d/30 - Loss: %.3f' % (i + 1, loss.data[0]))
    optimizer.step()

_ = model.eval()

Iter 1/30 - Loss: 624904.500
Iter 2/30 - Loss: 569460.375
Iter 3/30 - Loss: 520165.531
Iter 4/30 - Loss: 475969.125
Iter 5/30 - Loss: 436926.500
Iter 6/30 - Loss: 402379.844
Iter 7/30 - Loss: 371066.500
Iter 8/30 - Loss: 343971.281
Iter 9/30 - Loss: 319346.031
Iter 10/30 - Loss: 298302.281
Iter 11/30 - Loss: 279175.219
Iter 12/30 - Loss: 262366.906
Iter 13/30 - Loss: 247678.188
Iter 14/30 - Loss: 234838.250
Iter 15/30 - Loss: 223285.109
Iter 16/30 - Loss: 213321.125
Iter 17/30 - Loss: 204310.422
Iter 18/30 - Loss: 196279.406
Iter 19/30 - Loss: 189271.875
Iter 20/30 - Loss: 182959.125
Iter 21/30 - Loss: 177490.547
Iter 22/30 - Loss: 172539.188
Iter 23/30 - Loss: 168020.062
Iter 24/30 - Loss: 164160.641
Iter 25/30 - Loss: 160604.703
Iter 26/30 - Loss: 157486.391
Iter 27/30 - Loss: 154585.203
Iter 28/30 - Loss: 152077.406
Iter 29/30 - Loss: 149910.719
Iter 30/30 - Loss: 147754.734


In [107]:
predictions = model(x_test)
(predictions.mean() - y_test).abs().mean()

Variable containing:
 4.3721
[torch.FloatTensor of size 1]

In [71]:
def plot_model_and_predictions(model, plot_train_data=True):
    f, observed_ax = plt.subplots(1, 1, figsize=(4, 3))
    test_x = Variable(torch.linspace(0, 1, 51))
    observed_pred = model(test_x)

    def ax_plot(ax, rand_var, title):
        lower, upper = rand_var.confidence_region()
        if plot_train_data:
            ax.plot(train_x.data.numpy(), train_y.data.numpy(), 'k*')
        ax.plot(test_x.data.numpy(), rand_var.mean().data.numpy(), 'b')
        ax.fill_between(test_x.data.numpy(), lower.data.numpy(), upper.data.numpy(), alpha=0.5)
        ax.set_ylim([-3, 3])
        ax.legend(['Observed Data', 'Mean', 'Confidence'])
        ax.set_title(title)
    
    ax_plot(observed_ax, observed_pred, 'Observed Values (Likelihood)')
    
    return f

In [108]:
model.deep_net.parameters().next()[0:20]

Variable containing:
-6.9271e-01
-1.4959e-01
-2.9167e-02
 3.4251e-04
-1.1805e+00
 1.0660e+00
-1.5155e-03
-6.2615e-02
-3.5773e-02
-1.4900e-03
-4.4849e-01
-7.1094e-01
-5.2040e-01
-4.4658e-01
-3.3985e-01
 1.4964e-01
-5.7198e-03
 3.5688e-01
-3.4298e+00
-8.2014e-05
[torch.FloatTensor of size 20x1]

In [109]:
old_params

Variable containing:
-2.1611e-01
-3.5956e-04
-4.4229e-01
-4.5490e-02
 1.5626e-05
-4.0939e-02
 3.3181e-01
-1.5156e-01
-4.1293e-01
-5.4457e-05
 1.1744e-01
-1.8349e-01
-1.7113e-01
 2.2095e+00
-4.5600e-05
 1.1580e-01
-1.8889e-05
-5.3148e-01
-1.6299e-01
 1.0680e+00
[torch.FloatTensor of size 20x1]